# Sequential models

The `Sequential` model is a linear stack of layers.

You can create a `Sequential` model by passing a list of layer instances to the constructor:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_dim=784),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

You can also simply add layers via the `.add()` method:

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))

----

## Specifying the input shape

The model needs to know what input shape it should expect. For this reason, the first layer in a `Sequential` model (and only the first, because following layers can do automatic shape inference) needs to receive information about its input shape. There are several possible ways to do this:

- pass an `input_shape` argument to the first layer. This is a shape tuple (a tuple of integers or `None` entries, where `None` indicates that any positive integer may be expected). In `input_shape`, the batch dimension is not included.
- pass instead a `batch_input_shape` argument, where the batch dimension is included. This is useful for specifying a fixed batch size (e.g. with stateful RNNs).
- some 2D layers, such as `Dense`, support the specification of their input shape via the argument `input_dim`, and some 3D temporal layers support the arguments `input_dim` and `input_length`.

As such, the following three snippets are strictly equivalent:

In [ ]:
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

In [ ]:
model = Sequential()
model.add(Dense(32, batch_input_shape=(None, 784)))
# note that batch dimension is "None" here,
# so the model will be able to process batches of any size.

In [ ]:
model = Sequential()
model.add(Dense(32, input_dim=784))

And so are the following three snippets:

In [ ]:
from keras.layers.recurrent import LSTM

model = Sequential()
model.add(LSTM(32, input_shape=(10, 64)))

In [ ]:
model = Sequential()
model.add(LSTM(32, batch_input_shape=(None, 10, 64)))

In [ ]:
model = Sequential()
model.add(LSTM(32, input_length=10, input_dim=64))

Now you know enough to be able to define *almost* any model with Keras. For complex models that cannot be expressed via `Sequential` (and `Merge` layers, which we also did not cover), you can use [the functional API](/getting-started/functional-api-guide).


----

## Compilation

Before training a model, you need to configure the learning process, which is done via the `compile` method. It receives three arguments:

- an optimizer. This could be the string identifier of an existing optimizer (such as `rmsprop` or `adagrad`), or an instance of the `Optimizer` class. See: [optimizers](/optimizers).
- a loss function. This is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as `categorical_crossentropy` or `mse`), or it can be an objective function. See: [objectives](/objectives).
- a list of metrics. For any classification problem you will want to set this to `metrics=['accuracy']`. A metric could be the string identifier of an existing metric (only `accuracy` is supported at this point), or a custom metric function.

In [ ]:
# for a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# for a binary classification problem
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# for a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')

----

## Training

Keras models are trained on Numpy arrays of input data and labels. For training a model, you will typically use the `fit` function.

In [ ]:
# for a single-input model with 2 classes (binary):

model = Sequential()
model.add(Dense(1, input_dim=784, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# generate dummy data
import numpy as np
data = np.random.random((1000, 784))
labels = np.random.randint(2, size=(1000, 1))

# train the model, iterating on the data in batches
# of 32 samples
model.fit(data, labels, nb_epoch=10, batch_size=32)

`model.fit` has several optional parameters:

- `verbose`: how often the progress log is printed (0 for no log, 1 for progress bar logging, 2 for one line per epoch)
- `callbacks`: a list of callback objects that perform actions at certain events (see below)
- `validation_split`: splits the training data into training and validation sets. The value passed corresponds to the fraction of the data used for validation ( 
- `validation_data`: when you already have a validation set, pass a list in the format `[input, output]` here. Overrides `validation_split`.
- `shuffle` (default: `True`): shuffles the training data at each epoch.
- `class_weight` and `sample_weight`: used when you want to give different weights during training for certain classes or samples.

## Callbacks

While training a model, you often want to perform certain operations whenever certain events happen (for example, at the start or the end of every epoch or iteration). Keras supports several different callbacks that do things such as save your training history, save the model after each epoch (or whenever there is an improvement), and early stopping. Typical usage is as follows:


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
save_best = ModelCheckpoint('best_model.h5', save_best_only=True)
early_stop = EarlyStopping(patience=5)

history = model.fit(..., callbacks=[save_best, early_stop])

`history` is generated by the default callback, `keras.callbacks.History`, which is used every time `model.fit` is called so you never need to pass it as a callback.


# Saving and loading a model

Any given model will be composed of two separate parts:

- Its architecture, defined by your Keras code;
- the parameters, learned after training.

In order to reuse a previously trained model, Keras provides methods to serialize both the model architecture and its parameters separately. While it would be possible to use Python standard serialzation method (`pickle`), remember it is not necessarily portable across different Python versions (e.g., a file serialized using pickle on Python 2 will not load on Python 3 and vice-versa). Serializing the model architecture only works when the model does not use any `Lambda` layers; in that case, you have to reinstantiate the model programatically.

To serialize a model architecture, you can use the methods `model.to_json()` and `model.to_yaml()`. The only difference between both methods is the textual format used to serialize (YAML is meant to be more human-readable than JSON).

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(100,)))
model.add(Dense(10, activation='softmax'))

print(model.to_json())

In [ ]:
print(model.to_yaml())

Note: both methods only output a string, and it's up to you to write that string to a text file.

To save the model parameters, you can use the method `model.save_weights(filename)`. This saves *all* the model weights to an HDF5 file (which supports saving hierarchical data structures). This method already writes the file to the disk (as opposed to the methods described above). The `ModelCheckpoint` callback described above uses this function to save your model weights.

Having both the serialized architecture and parameters, you do not need the original code that generated the model to execute it anymore. This allows you to avoid the usual guesswork from iteratively running some experiments and fiddling with your code.

In [ ]:
from keras.models import model_from_json

model = model_from_json(json_string)
model.load_weights('my_weights_file.h5')